In [1]:
# Auxiliary Script
import numpy as np
import time
import inspect
import math
from typing import Tuple
from matplotlib import pyplot as plt

# import
if('models' not in sys.path):
    sys.path.append('models')

from Oracle import Oracle
from BOCS import BOCS
from SS_BOCS import SS_BOCS
from GeneticAlgo import GeneticAlgo
from RandomSearch import RandomSearch
from utils import get_alpha, expand

In [2]:
# Hyperparameters
N = 200
n = 180
d =  40
k =   2
sigma_eps = 1.0

np.random.seed(1534)

# Oracle
alpha  = get_alpha(d=d, k=k, p_zero=0.9, dist='gaussian', sigma=5)
f      = lambda x : sum([a_i * x_i for (a_i,x_i) in zip(alpha, [1]+x.tolist())])
o1     = Oracle(fun=f, alpha=alpha, x_dim=d, sigma=sigma_eps, max_eval=10**4) 

# Data
X      = np.random.binomial(n=1, p=0.5, size=n*d).reshape(n,-1)
y      = o1.f(X)

In [3]:
%%time

mode = 'max'

# Genetic Algorithm
ga1 = GeneticAlgo(n_init=20, n_total=N, oracle=o1, mode=mode, p_mut=0.05)
ga1.run()

# Random Search
rs1 = RandomSearch(d=d, n_total=N, oracle=o1, mode=mode)
rs1.run()

# BOCS
b1 = BOCS(n_sim=100, burnin=20, oracle=o1, d=d, k=2, afo='SDP', mode=mode)
b1.fit(X, y)
b1.run(N=N-len(X))

# SS-BOCS
s1 = SS_BOCS(oracle=o1, d=d, k=2, mode='max', afo='SDP')
s1.fit(X,y)
s1.run(N=N-len(X))

# moderate dimension, low sample size
print(f'MODE :  \hat{f}*_{mode}')
print('- - - - - - -')
print(f'GA      : {ga1.f_opt:.2f}')
print(f'RS      : {rs1.f_opt:.2f}')
print(f'BOCS    : {b1.f_opt:.2f}')
print(f'SS-BOCS : {s1.f_opt:.2f}\n')

MODE :  \hat<function <lambda> at 0x154904acde10>*_max
- - - - - - -
GA      : 57.88
RS      : 18.54
BOCS    : 93.68
SS-BOCS : 93.22

CPU times: user 2h 41min 55s, sys: 3h 3min 38s, total: 5h 45min 34s
Wall time: 7min 1s


In [4]:
np.diff(b1.timestamps).sum(), np.diff(s1.timestamps).sum()

(314, 69)

## 4 Scenarios
- sample size low/high: 50, 250
- dimension low/high:   10, 30

#### Sparsity `0.3`, Distribution `normal`

#### 1st scenario $d=10$, $N=50$

```
MODE :  max
- - - - - - -
GA   : 20.74
RS   : 12.43
BOCS* : 29.53
```

#### 2nd scenario $d=10$, $N=500$

```
MODE :  max
- - - - - - -
GA   : 20.74
RS   : 13.04
BOCS* : 29.73
```
#### 3rd scenario $d=20$, $N=50$

```
MODE :  max
- - - - - - -
GA   : 13.23
RS   : 31.38
BOCS* : 52.07
```

#### 4th scenario $d=20$, $N=500$

```
MODE :  max
- - - - - - -
GA    : 32.29
RS    : 38.67
BOCS* : 81.03
```

$N=250$, $d=20$
```
MODE :  max
- - - - - - -
GA      : 30.76
RS      : 18.31
BOCS    : 62.32
SS-BOCS : 61.66
```

In [ ]:
b1.statistical_model.model_estimate().round()

In [ ]:
alpha.round()